# Objective: Obtain out of fold predictions on the entire training set using cross validation and then using a mean average precision IoU metric, that closely resembles the competition metric, to improve validation

In [1]:
import numpy as np
import pandas as pd

# Prepare out of fold training predictions for implementation of MAP IoU matching competition evaluation description

Load oof predictions from CNN segmentation CV kernel https://www.kaggle.com/cchadha/cnn-segmentation-cv-with-oof-preds-on-train-set/notebook

In [2]:
oof_preds0 = pd.read_csv('preds/cnn_cv/oof_preds0.csv')
oof_preds1 = pd.read_csv('preds/cnn_cv/oof_preds1.csv')
oof_preds2 = pd.read_csv('preds/cnn_cv/oof_preds2.csv')

In [3]:
oof_preds0.head()

,patientId,PredictionString
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN
4,00436515-870c-4b36-a041-de91049b9ab4,0.5300612823673866 665 397 89 115 0.5212955657...


In [4]:
oof_preds1.head()

,patientId,PredictionString
0,628eea1d-9db2-4a99-acb3-0d846c3ac48f,NaN
1,629df578-2436-419b-9393-8ed83e2bda8a,NaN
2,629f59dc-5e5b-4c71-8201-308827e78873,NaN
3,62b4b084-06c5-413b-a365-ef2882ca98ab,NaN
4,62b8400b-e127-4874-984f-e34e08152af0,0.7613712313291122 142 422 266 333


In [5]:
oof_preds2.head()

,patientId,PredictionString
0,b05b4898-2003-4a43-a5bd-8de50200db4a,0.8005046991404918 526 319 231 373
1,b05deb36-6273-45fb-861a-c7058330c950,0.8482833748267393 216 282 259 408 0.820568150...
2,b0604ee6-5f21-4b14-98ca-3bedaccd4e10,NaN
3,b06a0fe5-dd0c-448e-bbbd-b773b17e5537,0.8287204544342874 216 216 258 423 0.789582330...
4,b0780603-efa4-453c-a51c-a23c4efedfc1,0.7361260794272702 601 321 244 343 0.754366174...


Read in training labels

In [6]:
df = pd.read_csv('data/stage_1_train_labels.csv')

In [7]:
df.head(20)

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1


Parse bounding box labels into correct format for Mean Average Precision IoU metric

In [8]:
df['bbox_target'] = (df['x'].astype(str) +
                    ' ' + 
                    df['y'].astype(str) +
                    ' ' +
                    df['width'].astype(str) +
                    ' ' +
                    df['height'].astype(str))

In [9]:
df.loc[:, 'bbox_target'] = df.loc[:, 'bbox_target'].map(lambda x: x.split(' '))

In [10]:
df.head(20)

,patientId,x,y,width,height,Target,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,"[264.0, 152.0, 213.0, 379.0]"
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1,"[562.0, 152.0, 256.0, 453.0]"
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1,"[323.0, 577.0, 160.0, 104.0]"
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1,"[695.0, 575.0, 162.0, 137.0]"


In [11]:
df = df.groupby(['patientId'], as_index = False)['bbox_target'].agg('sum')    

In [12]:
df.head()

,patientId,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]"
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]"
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]"
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]"
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256..."


Merge labels and oof preds

In [13]:
df = df.merge(oof_preds0, on = 'patientId', how = 'left')
df = df.merge(oof_preds1, on = 'patientId', how = 'left')
df = df.merge(oof_preds2, on = 'patientId', how = 'left')

In [14]:
df = df.fillna('')

Parse oof preds for MAP IoU

In [15]:
df.loc[:, 'bbox_pred'] = (df.loc[:, 'PredictionString'] +
                         ' ' +
                         df.loc[:, 'PredictionString_x'] +
                         ' ' +
                         df.loc[:, 'PredictionString_y'])

In [16]:
df = df.drop(['PredictionString','PredictionString_x', 'PredictionString_y'], axis=1)

In [17]:
df.head(20)

,patientId,bbox_target,bbox_pred
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...",0.5300612823673866 665 397 89 115 0.521295565...
5,00569f44-917d-4c86-a842-81832af98c30,"[nan, nan, nan, nan]",0.6062120299228089 652 513 181 127
6,006cec2e-6ce2-4549-bffa-eadfcd1e9970,"[nan, nan, nan, nan]",
7,00704310-78a8-4b38-8475-49f4573b2dbb,"[323.0, 577.0, 160.0, 104.0, 695.0, 575.0, 162...",
8,008c19e8-a820-403a-930a-bc74a4053664,"[nan, nan, nan, nan]",0.5044126939028502 334 475 12 10 0.5297812291...
9,009482dc-3db5-48d4-8580-5c89c4f01334,"[nan, nan, nan, nan]",


Stripping whitespace from PredictionString column

In [18]:
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].str.strip()

In [19]:
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].map(lambda x: x.split(' '))

In [20]:
df.head(10)

,patientId,bbox_target,bbox_pred
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[]
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",[]
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",[]
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",[]
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 665, 397, 89, 115, 0.5212..."
5,00569f44-917d-4c86-a842-81832af98c30,"[nan, nan, nan, nan]","[0.6062120299228089, 652, 513, 181, 127]"
6,006cec2e-6ce2-4549-bffa-eadfcd1e9970,"[nan, nan, nan, nan]",[]
7,00704310-78a8-4b38-8475-49f4573b2dbb,"[323.0, 577.0, 160.0, 104.0, 695.0, 575.0, 162...",[]
8,008c19e8-a820-403a-930a-bc74a4053664,"[nan, nan, nan, nan]","[0.5044126939028502, 334, 475, 12, 10, 0.52978..."
9,009482dc-3db5-48d4-8580-5c89c4f01334,"[nan, nan, nan, nan]",[]


In [21]:
def parse_scores(x):
    if len(x)!=1:
        scores = [x[k] for k in range(0,len(x),5)]
        for score in range(len(scores)):
            scores[score] = float(scores[score])
        return np.asarray(scores)

In [22]:
df.loc[:, 'bbox_scores'] = df.loc[:, 'bbox_pred'].map(parse_scores)

In [23]:
df.head()

,patientId,bbox_target,bbox_pred,bbox_scores
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[],None
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",[],None
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",[],None
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",[],None
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 665, 397, 89, 115, 0.5212...","[0.5300612823673866, 0.5212955657509237]"


In [24]:
def parse_bbox(x):
    if len(x)!=1:
        bbox = [int(x[k]) for k in range(0,len(x)) if k%5 != 0]
        return np.asarray(bbox).reshape(int(len(bbox)/4),4)

In [25]:
df.loc[:, 'bbox_preds'] = df.loc[:, 'bbox_pred'].map(parse_bbox)

In [26]:
df.head()

,patientId,bbox_target,bbox_pred,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[],None,None
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",[],None,None
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",[],None,None
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",[],None,None
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 665, 397, 89, 115, 0.5212...","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"


In [27]:
df = df.drop(['bbox_pred'], axis=1)

In [28]:
df.head(20)

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",None,None
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",None,None
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",None,None
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",None,None
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"
5,00569f44-917d-4c86-a842-81832af98c30,"[nan, nan, nan, nan]",[0.6062120299228089],"[[652, 513, 181, 127]]"
6,006cec2e-6ce2-4549-bffa-eadfcd1e9970,"[nan, nan, nan, nan]",None,None
7,00704310-78a8-4b38-8475-49f4573b2dbb,"[323.0, 577.0, 160.0, 104.0, 695.0, 575.0, 162...",None,None
8,008c19e8-a820-403a-930a-bc74a4053664,"[nan, nan, nan, nan]","[0.5044126939028502, 0.5297812291092973]","[[334, 475, 12, 10], [705, 562, 49, 26]]"
9,009482dc-3db5-48d4-8580-5c89c4f01334,"[nan, nan, nan, nan]",None,None


Edit NaN or None values to empty numpy arrays to fit MAP IoU metric implementation

In [29]:
df.loc[df['bbox_scores'].isnull(),['bbox_scores']] = df.loc[df['bbox_scores'].isnull(),'bbox_scores'].apply(lambda x: np.asarray([]))

In [30]:
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[],None
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",[],None
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",[],None
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",[],None
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"


In [31]:
df.loc[df['bbox_preds'].isnull(),['bbox_preds']] = df.loc[df['bbox_preds'].isnull(),'bbox_preds'].apply(lambda x: np.asarray([]))

In [32]:
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[],[]
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,"[nan, nan, nan, nan]",[],[]
2,00322d4d-1c29-4943-afc9-b6754be640eb,"[nan, nan, nan, nan]",[],[]
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,"[nan, nan, nan, nan]",[],[]
4,00436515-870c-4b36-a041-de91049b9ab4,"[264.0, 152.0, 213.0, 379.0, 562.0, 152.0, 256...","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"


In [33]:
def parse_target_str(x):
    if x[0] != 'nan':
        bbox = np.asarray([int(float(x[k])) for k in range(0,len(x))])
        return bbox.reshape(int(len(bbox)/4),4)

In [34]:
df.loc[:,'bbox_target'] = df.loc[:,'bbox_target'].map(parse_target_str)

In [35]:
df.loc[df['bbox_target'].isnull(),['bbox_target']] = df.loc[df['bbox_target'].isnull(),'bbox_target'].apply(lambda x: np.asarray([]))

In [36]:
df.head()

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,[],[],[]
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,[],[],[]
2,00322d4d-1c29-4943-afc9-b6754be640eb,[],[],[]
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,[],[],[]
4,00436515-870c-4b36-a041-de91049b9ab4,"[[264, 152, 213, 379], [562, 152, 256, 453]]","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"


# Find mean average precision IoU using implementation by chenyc15 https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein

In [37]:
# helper function to calculate IoU
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [38]:
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    if len(boxes_true) == 0 and len(boxes_pred) == 0:
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :]

        map_total = 0

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched:
                    fn += 1 # bt has no match, count as FN

            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp)
            map_total += m
    
    return map_total / len(thresholds)

In [39]:
df.head(20)

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,[],[],[]
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,[],[],[]
2,00322d4d-1c29-4943-afc9-b6754be640eb,[],[],[]
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,[],[],[]
4,00436515-870c-4b36-a041-de91049b9ab4,"[[264, 152, 213, 379], [562, 152, 256, 453]]","[0.5300612823673866, 0.5212955657509237]","[[665, 397, 89, 115], [742, 589, 38, 37]]"
5,00569f44-917d-4c86-a842-81832af98c30,[],[0.6062120299228089],"[[652, 513, 181, 127]]"
6,006cec2e-6ce2-4549-bffa-eadfcd1e9970,[],[],[]
7,00704310-78a8-4b38-8475-49f4573b2dbb,"[[323, 577, 160, 104], [695, 575, 162, 137]]",[],[]
8,008c19e8-a820-403a-930a-bc74a4053664,[],"[0.5044126939028502, 0.5297812291092973]","[[334, 475, 12, 10], [705, 562, 49, 26]]"
9,009482dc-3db5-48d4-8580-5c89c4f01334,[],[],[]


In [40]:
for row in range(len(df)):
    print(map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]))

None
None
None
None
0.0
0
None
0
0
None
None
None
0
0.041666666666666664
None
0.3333333333333333
None
0
None
0.5833333333333333
None
0.0
None
None
None
0.29166666666666663
None
None
0.29166666666666663
None
None
None
0.125
0
None
0
None
None
0.5
0.0
None
None
None
None
0
None
0
0
None
1.0
None
0
None
None
None
None
None
None
None
None
None
0.0
0
None
0
0
None
0.1875
0
None
0.0
0
None
0
None
0
None
None
None
None
0.0
None
None
0
0.3125
0
0.0
0.20833333333333331
0
None
0.0
0
0
None
None
0.0
None
0
None
None
1.0
None
0.375
None
0.0
0.0
None
None
None
None
None
0.0
None
None
None
None
None
None
0.0
0.0
0
0
0
None
None
0
0
0
None
None
None
None
None
None
None
None
0
None
0
0
None
0
0.25
0.375
None
None
0
None
None
None
None
None
0.29166666666666663
None
0.5416666666666666
0.0
None
None
None
0
None
0.16666666666666666
None
None
None
None
None
0.125
None
0.625
0.0
None
0.125
None
None
None
0
0
0.5833333333333333
0.0
None
None
0
0
None
0.0
0
None
None
None
0.0
0
None
0
None
0.7499999999999999


0
None
None
None
0.21875
None
None
0.0
0
0.0
None
0.16666666666666666
0
None
None
None
0
None
None
None
0
None
0.09375
None
0.0
None
0.125
None
None
None
0.0
None
0
0
None
0.7499999999999999
0.0
None
None
0.0
0.0
0.0
None
None
None
0
None
None
None
None
None
0
None
None
None
0.0
None
None
0
0.125
0
0.0
0.41666666666666674
None
0
None
None
0
None
0
0.1875
None
None
0
None
None
0.03125
0
0
0
0
0.5833333333333333
None
0.24999999999999997
0
None
None
0.0
None
0
0.041666666666666664
0
None
None
0
0
0.0
0
None
0.0
None
0.0
None
None
0
None
None
0
0.0
0.08333333333333333
None
0.0
None
None
None
None
0.6666666666666665
0.0
None
0
None
0
0
0
None
None
None
0.6666666666666665
0
None
0.0
None
None
0.125
None
0
0
0
0
0
None
None
0
0
0.0
0.0
None
None
None
None
0.0
0
0
None
0
None
None
0.125
None
0.4375
None
0.35416666666666663
None
None
None
0
None
0
None
None
0.0
0
0
None
0
None
None
None
None
None
None
None
None
None
0.16666666666666666
None
None
None
None
0.0
0
0.16666666666666666
None
None
0
N

0
0
None
None
None
0
0
None
None
None
None
0
None
None
0
0
None
None
0.4166666666666667
None
None
0
None
None
None
0
None
None
None
None
None
None
None
None
None
None
None
None
0
None
None
0
None
None
0.0
None
0
0.0
None
None
None
None
None
0.0
0.5833333333333333
0
None
0
None
None
0
None
0
None
None
0.3333333333333333
0.3333333333333333
0
0
0.0
0.7083333333333333
None
None
None
None
0.7916666666666666
None
None
None
None
None
None
None
None
None
None
None
0
None
0
0
None
None
0
None
0.0
None
0
None
None
None
None
None
None
0
None
None
0.0
None
None
None
None
None
None
None
0
None
None
None
0
None
None
0
None
None
None
None
None
0.0
None
None
0
None
None
None
None
None
None
0
0
None
None
0
0
None
0
None
0
None
None
0
None
None
0
None
None
None
None
0.6666666666666666
0
None
None
None
None
None
0
0.0
0.1875
None
0
None
0
None
0
0
0.5416666666666666
None
0
0
None
None
None
None
None
None
None
None
None
0.20833333333333331
0
None
None
None
None
None
None
None
None
0.375
None
None
None
Non

0
0
None
1.0
None
None
0
0.0
None
None
None
0
None
None
None
None
None
None
None
0
None
0
0.0
0
None
None
0.0
0.0
0
None
None
None
0
None
None
None
0
None
None
0.0
None
None
None
0
None
None
0
None
None
None
None
None
None
None
None
None
None
0
None
None
None
0
0
0
None
None
0
None
0
None
0
0
None
0
None
None
0.875
0.0
0
None
0.3125
None
None
None
0
0.0625
None
None
0
None
None
None
None
None
None
None
None
None
0
None
None
None
None
None
0.16666666666666666
None
None
0.041666666666666664
None
0
None
None
None
None
None
0.16666666666666666
None
None
0.0
0
None
None
None
0
None
None
None
None
None
0.0
0.25
None
0
None
None
0
None
None
None
None
None
None
None
None
None
0.4375
0.20833333333333331
None
None
None
0.4166666666666667
0
None
None
None
0.0
None
None
None
None
0.29166666666666663
None
None
0
0.16666666666666666
None
None
0
None
None
0
0
None
None
None
None
None
None
None
None
None
0.09375
0
None
None
None
0
None
0.08333333333333333
None
None
0
None
None
None
None
0.0
None
None


0.5416666666666666
None
None
0.0
0.0
None
None
0.375
None
0
None
0.875
None
0
0
None
None
None
0
None
None
None
0
0
0.6666666666666666
None
None
0.5
None
None
None
None
0
None
None
None
None
None
None
None
None
None
0
0
None
None
None
None
0
None
None
0.2916666666666667
0.125
0
None
None
None
None
None
0.3125
0.7499999999999999
0.0
0.5416666666666666
0
0.4166666666666667
None
None
0.0
None
1.0
None
0.0
None
None
None
0
0
None
0.0
0.0625
0.0625
0
0.0
None
None
None
0.041666666666666664
0.0
None
0
0.5416666666666666
None
None
0.0
None
None
0.75
0
None
0
None
None
None
None
0
None
None
None
None
None
0.0
None
0.0
0.0
0.5
0
None
None
0.125
None
0.0
None
None
None
0
None
None
0.625
None
None
0
0
None
0.6249999999999999
None
None
0
None
None
0.29166666666666663
None
0
None
None
0
0.08333333333333333
0
0.7916666666666666
None
0
0
0.0
None
None
None
None
None
None
0
0
0.25
None
0.08333333333333333
None
None
0
None
0
None
None
0.0
None
None
None
0.7916666666666666
0.375
None
None
None
None
None

...

None
0.24999999999999997
0.25
None
None
0.0
0
None
None
0.0
None
None
None
None
0
None
0.625
None
0.7916666666666666
None
None
None
None
0
0.08333333333333333
None
None
None
0
0
None
0
0
None
None
0.25
0
0
None
None
None
None
None
None
None
None
None
None
None
None
None
1.0
None
None
None
None
None
None
0
0.8333333333333333
0
0
None
0.75
None
None
0.2916666666666667
None
0.24999999999999997
None
0.125
0.37500000000000006
None
None
0.29166666666666663
0
None
None
None
None
None
None
None
None
None
0.25
0
None
0.4166666666666667
0
None
None
None
None
None
None
None
None
None
None
None
None
None
0.5
None
None
0
None
None
None
None
None
0.5
None
None
None
None
0.0
None
None
None
None
None
None
None
0.5000000000000001
0.0
None
None
0
0.3333333333333333
None
None
None
None
None
None
None
None
None
None
None
None
None
0
None
0
0
0
0.0
0.9166666666666666
None
None
0.5000000000000001
0.4375
None
None
0.0625
0
0
None
None
None
0.875
None
None
None
None
None
None
0.0
None
0
0
None
0
None
None
0.0

None
None
None
0
1.0
None
0.6249999999999999
None
0.0
None
0
0.0
None
None
0
0.0
0
None
0.6666666666666666
None
None
0.5833333333333333
0.03125
0.6666666666666666
None
None
0.24999999999999997
0.6666666666666666
None
0
None
0.0
None
0.4375
None
None
None
None
None
None
0
None
None
None
0
None
None
0.2916666666666667
None
None
0
0.375
None
0.0
None
None
None
0
None
0
0.8333333333333333
None
0
None
0.5416666666666666
0.6249999999999999
None
None
None
0.125
None
None
None
None
None
0
0.3645833333333333
None
None
None
None
0.0625
0
None
0
None
0
None
None
None
None
None
0.25
None
None
None
0
0
None
0
0.375
None
None
0
None
0
None
0.25
0
None
None
None
None
None
0
None
None
None
0.8333333333333333
0.6666666666666665
None
0.0
0
0.24999999999999997
None
0
0.0
None
0
None
0
0
0.9166666666666666
0.125
None
0.5
None
None
1.0
None
None
0
0
None
None
None
0.7083333333333333
None
0.3333333333333333
0
0.6666666666666665
None
0
0.15625
None
None
0
0
None
None
None
None
0
None
None
None
0.125
0.0625
N

None
0.45833333333333337
None
None
None
None
0
None
None
None
None
1.0
0.5416666666666666
None
0
0
None
None
None
None
0
None
None
None
None
None
0.16666666666666666
None
None
0.5
None
None
None
None
None
None
None
0.25
None
0.375
0
None
None
0
None
0
None
None
None
0
0
0
None
0
None
None
0
0
None
None
None
None
0
None
None
None
None
0
0
None
0
None
None
None
0
0
None
None
0.4583333333333333
0.0
None
None
0
None
None
0.0
None
None
None
None
0.5
None
None
None
0
0.4166666666666667
None
0.24999999999999997
None
None
None
0.4375
0
None
None
None
0
None
None
None
None
None
None
None
0
0.0625
None
None
None
0
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
0
None
None
None
None
0
0
None
0
None
None
None
None
None
None
None
0.3333333333333333
None
0.3125
None
0
None
None
None
None
0.1875
None
0.45833333333333337
0
None
None
None
None
None
None
None
None
None
None
None
None
None
0
None
None
0
None
None
None
None
None
None
None
None
0
No

0.0
None
None
None
None
None
0.16666666666666666
0
None
None
0
None
None
0
0
0
0.0
0
None
None
None
None
None
None
0
None
None
None
0
0
None
None
None
0
None
0
None
None
None
0
None
0
None
0
None
0
None
None
0
None
None
0
None
None
None
None
None
None
0
None
0
0
0
0
None
0
0
None
0
0.0
None
0.0
None
None
None
0.0
0
None
None
None
None
None
None
None
None
0.0625
None
None
None
None
None
None
None
None
None
None
0
None
0.15625
None
0
None
0
None
None
None
None
0.0
0
None
None
None
None
None
None
0.0
0
None
0.0
None
None
0
0
None
None
None
None
None
None
None
None
None
None
0.0
0
None
0.0
None
None
None
None
None
None
0.0625
0.0
None
None
None
None
0
0
None
None
0.0
0.29166666666666663
None
0
None
0
0.16666666666666666
0.1
0
0
None
None
None
None
0
None
None
None
None
0
0
0.29166666666666663
None
None
None
None
None
None
None
None
0
None
None
None
0
0
None
None
None
None
None
None
None
None
0.0
None
0.0
None
0.0
0
None
0.1875
0
None
None
0
None
0
None
None
0
None
None
None
0
0.0
None
0.24

None
None
None
None
None
None
None
0.375
None
0.3125
0
0
0
0
0
None
None
None
None
None
None
0
None
None
None
None
None
None
None
None
None
0
None
None
None
0.0
None
None
None
None
None
None
None
None
None
0
None
None
0
0.0875
0
None
None
None
None
None
0.5
None
0
None
0.16666666666666666
None
None
None
None
None
None
None
0
None
None
0
None
None
None
None
None
None
0
None
0
None
None
None
None
0
None
None
0.125
None
0
None
None
None
None
None
None
0
None
None
None
None
0
None
None
None
None
None
0
0
None
0.25
0.125
None
None
None
None
None
0
None
0.0
None
0
0
None
0.0
0
None
0.4375
None
None
None
None
None
None
0
None
None
None
None
None
None
0
None
None
None
None
None
None
None
None
0.5
None
None
None
None
0.125
None
None
0
None
None
0
None
None
None
0.08333333333333333
None
None
None
None
None
None
None
None
None
None
None
None
0.1875
None
None
None
None
None
None
None
0
None
0.0
None
None
None
None
None
None
0.041666666666666664
None
None
None
0.375
None
None
None
None
None
0
None


In [41]:
map_scores = [x for x in [map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]) for row in range(len(df))] if x is not None]

In [42]:
np.mean(map_scores)

0.10302512510216258